In [1]:
#import libraries
import boto3
import csv
import json
import time

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
# file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
# folder = 'C:/Users/jordi/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_wri.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
# s3BucketName = "wri-testing"
s3BucketName = "wri-nlp-policy"
# region = 'eu-central-1'
region = "us-east-1"

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [ ]:
prefix = "english_documents/raw_pdf/"
i = 0
for object in s3.Bucket(s3BucketName).objects.all().filter(Prefix=prefix):
    srcKey = object.key
    if ".txt" in srcKey:
        print(i)
        newKey = srcKey.replace(".txt", ".pdf")
        copySource = s3BucketName + '/' + srcKey         
        s3.Object(s3BucketName, newKey).copy_from(CopySource=copySource)
        s3.Object(s3BucketName, srcKey).delete()
    i += 1

In [3]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
#     print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
#         print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
#     print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
#             print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [4]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
#     print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
#         print(resultPage)
        try:
            for item in resultPage["Blocks"]:
                if item["BlockType"] == "LINE":
                    policy = policy + item["Text"] + "\n"
#                     print ('\033[94m' + item["Text"] + '\033[0m')
        except:
            f = open("../output/log_file.txt", 'a')
            f.write(f"The file {key} could not be converted\n")
            f.close()
            
            print(f"The file {key} could not be converted")
    return policy

In [6]:
keyword = "0bc17448527c7798e84937eb897bdaa82c3c4a3c.pdf"
prefix = "spanish_documents/raw_pdf"
i = 0


for obj in s3.Bucket(s3BucketName).objects.all().filter(Prefix = prefix):#.:
    if ".pdf" in obj.key: #and i > 48: lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        key = obj.key.replace(".pdf", ".txt")#replace("raw_pdf", "text_files").
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        if policy != "":
            s3.Object(s3BucketName, key).put(Body = policy)
#         save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

1 ** english_documents/text_files/HSSC/new/000e7c51d84c35a641edfbbdf973be18ac5a221f.pdf
2 ** english_documents/text_files/HSSC/new/0084621d4aba883702f73c91a893650326e93494.pdf
3 ** english_documents/text_files/HSSC/new/037ca13714e6c2d460e08f7387e376a0de1204e6.pdf
4 ** english_documents/text_files/HSSC/new/03c5e81d26e5970fcf3368213c5a0ec3426ebef2.pdf
5 ** english_documents/text_files/HSSC/new/047fc8bc16b8389755df5ce05d5807c86995aad5.pdf
6 ** english_documents/text_files/HSSC/new/062774da183f5ffa5819b558325a0a5ed5d78d3c.pdf
7 ** english_documents/text_files/HSSC/new/069865908029679b02104a049fdeed07addc463a.pdf
8 ** english_documents/text_files/HSSC/new/07f58233882846915a5c8f750671ac781ab4f50f.pdf
9 ** english_documents/text_files/HSSC/new/0bbd936eb6325d97f2890a153d98ff97acb38fc6.pdf
10 ** english_documents/text_files/HSSC/new/0f659dfe706ea231c71e2e4778116e7754485882.pdf
11 ** english_documents/text_files/HSSC/new/11f6495243d9a5d7b81fb7ad204c780682634585.pdf
12 ** english_documents/text_f

97 ** english_documents/text_files/HSSC/new/8688318b8965668b46c17bc9df0b0016f2b35f1c.pdf
98 ** english_documents/text_files/HSSC/new/873f2e352d4575fd293cf14a18a4ab74c7b3544d.pdf
99 ** english_documents/text_files/HSSC/new/879f6312f99dec98c52363f78d3a7d7e6db6cabf.pdf
100 ** english_documents/text_files/HSSC/new/8d1d94c39f850a2255eb5675c203210e9830c87e.pdf
101 ** english_documents/text_files/HSSC/new/8ef7fe2aad53cb2f4ff4f874368189a68569b7e0.pdf
102 ** english_documents/text_files/HSSC/new/900a67b81b34f80fb7f1e544a97236a726589d16.pdf
103 ** english_documents/text_files/HSSC/new/932ecb2a8473ee39cbae2b32e08b87707cabeb20.pdf
104 ** english_documents/text_files/HSSC/new/961bf1c67d341a33c0742dbbe32e74d5303f49d2.pdf
105 ** english_documents/text_files/HSSC/new/968ac0c404ab53bdb730b5499f57caab6a8b41ff.pdf
106 ** english_documents/text_files/HSSC/new/97045d3b327166abf1d7187420679275f98b4caa.pdf
107 ** english_documents/text_files/HSSC/new/9815b5a6817a0427c5c4467c3d964741013535a1.pdf
108 ** englis

In [ ]:
policy